In [20]:
import pandas as pd
import datetime 
import time
import numpy as np
from matplotlib import pyplot as plt, rcParams

import ast
import io
import webbrowser
from random import shuffle
from bson import ObjectId                                                                                                  

import brtdevkit
from brtdevkit.core.db import DBConnector
from brtdevkit.core.db.db_filters import *  # We need this for pre-defined filters, e.g., ProjectFilter, DatetimeFilter

def get_specific_fields(df, artifact = 'nrg'):
    """
    Takes a dataframe with specific [grower, farm, operating_field_name] combinations and retreives document records from 
    Aletheia matching just those fields. Useful when you  need a collection of fields but don't want to query the entire record.
    :param: df: a pandas dataframe with [grower, farm, operating_field_name] columns.
    :param: artifact: The type of artifact you want to return. Default is 'nrg'
     """
    db = DBConnector()
    filters = {'grower': df['grower'][0], 'farm': df['farm'][0],'operating_field_name': df['operating_field_name'][0]}
    img_filters = {'project_name': 'shasta', **filters, 'artifacts.kind':artifact}
    master_df = db.get_documents_df('image', img_filters, limit=None)
    for n in range(len(df[1:])):
        filters = {'grower': str(df['grower'][n]), 'farm': str(df['farm'][n]),'operating_field_name': str(df['operating_field_name'][n])}
        img_filters = {'project_name': 'shasta', **filters, 'artifacts.kind':artifact}
        field_df = db.get_documents_df('image', img_filters, limit=None)
        if len(field_df)==0:
            print(f'Could not find field: { df.operating_field_name[n]} from grower: {df.grower[n]}')
        master_df = pd.concat([master_df, field_df], axis=0)
    print(f'Query returned {len(master_df)} records.')
    return master_df

In [21]:
low_solar_angles = pd.read_csv('low_solar_angle_fields.csv')

df = get_specific_fields(low_solar_angles)
df.has_human_annotation.value_counts()

Could not find field: 30 from grower: steve fullen
Query returned 59602 records.


False    55506
True      4096
Name: has_human_annotation, dtype: int64

In [11]:
df.shape

(4111, 63)

In [16]:
df.grower_farm_field.value_counts()

AttributeError: 'DataFrame' object has no attribute 'grower_farm_field'